In [2]:
#!pip install s3fs

   ---------------------------------------- 0.0/176.9 kB ? eta -:--:--
   ------------- -------------------------- 61.4/176.9 kB 3.4 MB/s eta 0:00:01
   -------------------------------- ------- 143.4/176.9 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 176.9/176.9 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/76.6 kB ? eta -:--:--
   ---------------------------------------- 76.6/76.6 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/370.7 kB ? eta -:--:--
   ------------------------------------ --- 337.9/370.7 kB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 370.7/370.7 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   - -------------------------------------- 0.6/12.2 MB 17.5 MB/s eta 0:00:01
   ---- ----------------------------------- 1.5/12.2 MB 18.8 MB/s eta 0:00:01
   -------- ------------------------------- 2.7/12.2 MB 21.1 MB/s eta 0:00:01
   ----

In [1]:
from kafka import KafkaConsumer, KafkaProducer
from time import sleep
from json import dumps,loads
import json
from s3fs import S3FileSystem
import pandas as pd

In [2]:
# Membaca file saham yang akan digunakan
df = pd.read_csv('cuaca_realtime_sby_mlg.csv', delimiter=';')
df

,name,region,country,lat,lon,tz_id,localtime,last_updated,temp_c,temp_f,...,wind_kph,wind_degree,pressure_in,precip_in,humidity,cloud,feelslike_c,uv,gust_mph,gust_kph
0,Surabaya,East Java,Indonesia,-7.25,112.75,Asia/Jakarta,21/05/2024 08:45,21/05/2024 08:45,30.0,86.0,...,9.0,100,29.83,0.0,70,25,37.6,8.0,10.1,16.2
1,Malang,East Java,Indonesia,-8.00,114.14,Asia/Jakarta,21/05/2024 08:44,21/05/2024 08:30,19.7,67.4,...,4.0,176,29.89,0.0,85,85,19.7,5.0,4.3,6.8
2,Surabaya,East Java,Indonesia,-7.25,112.75,Asia/Jakarta,21/05/2024 08:45,21/05/2024 08:45,30.0,86.0,...,9.0,100,29.83,0.0,70,25,37.6,8.0,10.1,16.2
3,Malang,East Java,Indonesia,-8.00,114.14,Asia/Jakarta,21/05/2024 08:47,21/05/2024 08:45,19.7,67.4,...,4.0,176,29.89,0.0,85,85,19.7,5.0,4.3,6.8
4,Surabaya,East Java,Indonesia,-7.25,112.75,Asia/Jakarta,21/05/2024 08:48,21/05/2024 08:45,30.0,86.0,...,9.0,100,29.83,0.0,70,25,37.6,8.0,10.1,16.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,Malang,East Java,Indonesia,-8.00,114.14,Asia/Jakarta,21/05/2024 17:02,21/05/2024 17:00,21.6,70.9,...,3.6,147,29.81,0.0,85,68,21.6,6.0,6.7,10.8
992,Surabaya,East Java,Indonesia,-7.25,112.75,Asia/Jakarta,21/05/2024 17:02,21/05/2024 17:00,31.0,87.8,...,19.1,80,29.71,0.0,66,25,35.3,8.0,16.3,26.3
993,Malang,East Java,Indonesia,-8.00,114.14,Asia/Jakarta,21/05/2024 17:05,21/05/2024 17:00,21.6,70.9,...,3.6,147,29.81,0.0,85,68,21.6,6.0,6.7,10.8
994,Surabaya,East Java,Indonesia,-7.25,112.75,Asia/Jakarta,21/05/2024 17:06,21/05/2024 17:00,31.0,87.8,...,19.1,80,29.71,0.0,66,25,35.3,8.0,16.3,26.3


In [3]:
def produce_weather_data():
    producer = KafkaProducer(bootstrap_servers='3.107.68.101:9092')
    csv_file = 'cuaca_realtime_sby_mlg.csv'
    df = pd.read_csv(csv_file)
    
    for index, row in df.iterrows():
        data = row.to_dict()  
        producer.send('data-cuaca-real-time', value=str(data).encode('utf-8'))
        producer.flush()  
    
    producer.close()

produce_weather_data()